In [1]:
import datetime as dt
import json
import pandas as pd
import psycopg2

from concurrent.futures.thread import ThreadPoolExecutor

In [2]:
conn = psycopg2.connect(dbname='skyserver_dr16_2020_11_30', 
                        user='postgres',
                        host='testbed.inode.igd.fraunhofer.de', 
                        password='vdS83DJSQz2xQ', 
                        port=18001)

tables = [
    "galspecline",
    "neighbors",
    "photoobj",
    "specobj",
    "spplines"
]

In [3]:
result = {}
bin_count = 10


def get_value(index, column, table, bin_size, conn):
    value = pd.read_sql(
        f"select {column} from {table} where {column} is not null order by {column} offset { index*bin_size  } limit 1;",
        conn,
    ).iloc[0][column]
    return value


for table in tables:
    columns = list(
        pd.read_sql(
            f"SELECT column_name  FROM INFORMATION_SCHEMA. COLUMNS WHERE TABLE_NAME = '{table}' and data_type in ('date', 'smallint', 'integer', 'decimal', 'numeric', 'real', 'double precision', 'smallserial', 'serial', 'bigserial', 'bigint', 'real');",
            conn,
        )["column_name"]
    )
    for column in columns:
        if not "_id" in column:
            item_count = pd.read_sql(
                f"select count(*) from {table} where {column} is not null;", conn
            ).iloc[0]["count"]
            bin_size = (item_count // bin_count) - 1
            distinct_values = pd.read_sql(
                f"select count(distinct {column}) from {table} ;", conn
            ).iloc[0]["count"]
            null_count = pd.read_sql(
                f"select count(*)  from {table} where {column} is null", conn
            ).iloc[0]["count"]
            print(
                f"Column: {column}, {distinct_values} distinct val, {null_count} null values"
            )
            # if null_count == 0:
            bins = []
            if distinct_values > 50:
                futures = []
                values = []
                with ThreadPoolExecutor(max_workers=11) as executor:
                    for i in range(11):
                        futures.append(
                            executor.submit(get_value, i, column, table, bin_size, conn)
                        )

                    for future in futures:
                        values.append(future.result())

                    values.sort()
                    if type(values[0]) == dt.date:
                        values = map(lambda x: x.isoformat(), values)
                    else:
                        values = map(lambda x: float(x), values)
                    for index, value in enumerate(values):
                        if index > 0:
                            bins.append((last_value, value))
                        last_value = value
            else:
                bins = list(
                    pd.read_sql(
                        f"select distinct {column} from {table} where {column} is not null order by {column};",
                        conn,
                    )[column]
                )
                if type(bins[0]) == dt:
                    bins = map(lambda x: x.isoformat(), bins)

            result[f"{table}.{column}"] = bins

with open("bins.json", "w") as fp:
    json.dump(result, fp, indent=1)

/var/folders/d6/7jdh7rbj2f59v10kcc72rsym0000gn/T/ipykernel_19918/4072846926.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(
/var/folders/d6/7jdh7rbj2f59v10kcc72rsym0000gn/T/ipykernel_19918/4072846926.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  item_count = pd.read_sql(
/var/folders/d6/7jdh7rbj2f59v10kcc72rsym0000gn/T/ipykernel_19918/4072846926.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  distinct_values = pd.read_sql(
/var/folders/d6/7jdh7rbj2f59v10kcc72rsym0000gn/T/ipykernel_19918/4072846

Column: specobjid, 852096 distinct val, 0 null values


/var/folders/d6/7jdh7rbj2f59v10kcc72rsym0000gn/T/ipykernel_19918/4072846926.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  item_count = pd.read_sql(
/var/folders/d6/7jdh7rbj2f59v10kcc72rsym0000gn/T/ipykernel_19918/4072846926.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  distinct_values = pd.read_sql(
/var/folders/d6/7jdh7rbj2f59v10kcc72rsym0000gn/T/ipykernel_19918/4072846926.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  null_count = pd.read_sql(
/var/folders/d6/7jdh7rbj2f59v10kcc72rsym0000gn/T/ipykernel_

Column: specobjid, 852096 distinct val, 0 null values


/var/folders/d6/7jdh7rbj2f59v10kcc72rsym0000gn/T/ipykernel_19918/4072846926.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  item_count = pd.read_sql(
/var/folders/d6/7jdh7rbj2f59v10kcc72rsym0000gn/T/ipykernel_19918/4072846926.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  distinct_values = pd.read_sql(
/var/folders/d6/7jdh7rbj2f59v10kcc72rsym0000gn/T/ipykernel_19918/4072846926.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  null_count = pd.read_sql(


...

Column: sii_6731_flux_err, 782795 distinct val, 0 null values


/var/folders/d6/7jdh7rbj2f59v10kcc72rsym0000gn/T/ipykernel_19918/4072846926.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  value = pd.read_sql(
/var/folders/d6/7jdh7rbj2f59v10kcc72rsym0000gn/T/ipykernel_19918/4072846926.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  item_count = pd.read_sql(
/var/folders/d6/7jdh7rbj2f59v10kcc72rsym0000gn/T/ipykernel_19918/4072846926.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  distinct_values = pd.read_sql(
/var/folders/d6/7jdh7rbj2f59v10kcc72rsym0000gn/T/ipykernel_19918/

Column: sii_6731_inst_res, 612243 distinct val, 0 null values


/var/folders/d6/7jdh7rbj2f59v10kcc72rsym0000gn/T/ipykernel_19918/4072846926.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  value = pd.read_sql(


KeyboardInterrupt: 